In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv


In [53]:
import pandas as pd
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
sample_submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [59]:
oct_trans=sales_train[sales_train["date"].str.endswith("10.2015")]
oct_sales = oct_trans.groupby(['shop_id','item_id'])['item_cnt_day'].sum()

In [63]:
nov_trans=sales_train[sales_train["date"].str.endswith("11.2014")]
nov_sales = nov_trans.groupby(['shop_id','item_id'])['item_cnt_day'].sum()

In [64]:
merged=pd.merge(test,oct_sales,on=['shop_id','item_id'],how="outer")
#merged=merged[merged['ID']!=np.nan]
merged=merged.dropna(subset=['ID'])
merged=merged.replace(np.nan,0)
merged['item_cnt_month']=merged['item_cnt_day'].clip(lower=0,upper=20)
merged=merged.drop(['shop_id','item_id','item_cnt_day'],axis=1)
merged['ID']=merged['ID'].astype('int')
merged.to_csv("october_sales.csv",index=False)
merged
# print(merged.shape)
# sample_submission.shape

,ID,item_cnt_month
0,0,1.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
214195,214195,0.0
214196,214196,0.0
214197,214197,0.0
214198,214198,0.0
